## Basic Prompting

In [1]:
prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Context: {context}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

# This is equivalent of giving SystemPromptTemplate and HumanPromptTemplate as an input as we saw in LangChain Prompt Basics
prompt_template = ChatPromptTemplate.from_messages([
    ("system", prompt),
    ("user", "{query}"),
])

In [3]:
prompt_template.input_variables

['context', 'query']

Prompt template expects us to give all these input variable information when we query the LLM

In [4]:
prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

In [ ]:
from langchain_ollama import ChatOllama
# Low temperature as we do not want model to hallucinate by being creative for the task we are makling it do
llm = ChatOllama(model="deepseek-r1:1.5b", temperature=0.0)

In [6]:
pipeline = prompt_template | llm

In [7]:
context = """Aurelio AI is an AI company developing tooling for AI
engineers. Their focus is on language AI with the team having strong
expertise in building AI agents and a strong background in
information retrieval.

The company is behind several open source frameworks, most notably
Semantic Router and Semantic Chunkers. They also have an AI
Platform providing engineers with tooling to help them build with
AI. Finally, the team also provides development services to other
organizations to help them bring their AI tech to market.

Aurelio AI became LangChain Experts in September 2024 after a long
track record of delivering AI solutions built with the LangChain
ecosystem."""

query = "what does Aurelio AI do?"

In [9]:
pipeline.invoke({"query": query, "context": context}).content

"<think>\nOkay, so I need to figure out what Aurelio AI does based on the context provided. Let me read through the context again carefully.\n\nThe context says that Aurelio AI is an AI company focused on language AI. They have a team with expertise in building AI agents and strong background in information retrieval. They are behind several open-source frameworks like Semantic Router and Semantic Chunkers, and they have an AI Platform for engineers to build with LangChain. Additionally, they provide development services to other organizations to help them bring their AI tech to market.\n\nSo, putting that together, Aurelio AI seems to be involved in language-related AI solutions. They use open-source frameworks which suggests they might work on creating or improving those tools. Their focus on information retrieval indicates they handle aspects related to retrieving and processing data efficiently. They also have a platform for engineers, so maybe they provide support or services tail

## Few shot Prompting

In [11]:
from langchain.prompts import FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

examples = [
    {"input": "Here is query #1", "output": "Here is the answer #1"},
    {"input": "Here is query #2", "output": "Here is the answer #2"},
    {"input": "Here is query #3", "output": "Here is the answer #3"},
]

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: Here is query #1
AI: Here is the answer #1
Human: Here is query #2
AI: Here is the answer #2
Human: Here is query #3
AI: Here is the answer #3


In [12]:
prompt_template

ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})])

In [ ]:
from IPython.display import display, Markdown

new_system_prompt = """
Answer the user's query based on the context below.                 
If you cannot answer the question using the
provided information answer with "I don't know".

Always answer in markdown format. When doing so please
provide headers, short summaries, follow with bullet
points, then conclude.

Context: {context}
"""

prompt_template.messages[0].prompt.template = new_system_prompt

examples = [
    {
        "input": "Can you explain gravity?",
        "output": (
            "## Gravity\n\n"
            "Gravity is one of the fundamental forces in the universe.\n\n"
            "### Discovery\n\n"
            "* Gravity was first discovered by Sir Isaac Newton in the late 17th century.\n"
            "* It was said that Newton theorized about gravity after seeing an apple fall from a tree.\n\n"
            "### In General Relativity\n\n"
            "* Gravity is described as the curvature of spacetime.\n"
            "* The more massive an object is, the more it curves spacetime.\n"
            "* This curvature is what causes objects to fall towards each other.\n\n"
            "### Gravitons\n\n"
            "* Gravitons are hypothetical particles that mediate the force of gravity.\n"
            "* They have not yet been detected.\n\n"
            "**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.\n\n"
        )
    },
    {
        "input": "What is the capital of France?",
        "output": (
            "## France\n\n"
            "The capital of France is Paris.\n\n"
            "### Origins\n\n"
            "* The name Paris comes from the Latin word \"Parisini\" which referred to a Celtic people living in the area.\n"
            "* The Romans named the city Lutetia, which means \"the place where the river turns\".\n"
            "* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.\n\n"
            "**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.\n\n"
        )
    }
]


few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

out = few_shot_prompt.format()
prompt_template = ChatPromptTemplate.from_messages([
    ("system", new_system_prompt),
    few_shot_prompt,
    ("user", "{query}"),
])
pipeline = prompt_template | llm
# Few Shot Prompting
out = pipeline.invoke({"query": query, "context": context}).content
out

"<think>\nAlright, so I'm trying to figure out what Aurelio AI does. From the context provided earlier, I know that Aurelio AI is an AI company focused on language AI with a strong team expertise in building AI agents and having a solid background in information retrieval. They also have several open-source frameworks like Semantic Router and Semantic Chunkers, and they offer an AI Platform to help engineers build with AI. Additionally, they provide development services to other organizations to bring their AI tech to market.\n\nNow, the user is asking specifically about what Aurelio AI does. I need to break this down into key areas of focus based on the information given. First, since they're focused on language AI, that's a primary area. They likely have tools and platforms designed for natural language processing tasks like translation, text generation, summarization, etc.\n\nNext, their team expertise in building AI agents suggests they develop models and systems that can perform c

In [18]:
out = """
Aurelio AI is an AI company focused on language AI with several key areas of focus:\n\n1. **Language Processing**: \n   - Develops tools and platforms for natural language processing (NLP) tasks such as translation, text generation, summarization, and more.\n\n2. **AI Platform Development**:\n   - Offers a centralized environment where engineers can develop, test, and deploy AI models efficiently, including features like real-time collaboration and automated testing.\n\n3. **Information Retrieval**:\n   - Manages data retrieval tasks effectively, optimizing search algorithms and indexing large datasets for quick information access.\n\n4. **Open-Source Framework Integration**:\n   - Utilizes frameworks like Semantic Router and Semantic Chunkers to enhance their AI Platform's capabilities in processing and understanding text data through components such as tokenization, parsing, and semantic analysis.\n\n5. **Tailored Development Services**:\n   - Provides organizations with customizations of their platforms and integration opportunities for new technologies, enabling them to bring advanced AI solutions to market effectively.\n\nIn summary, Aurelio AI focuses on language processing, AI platform development, information retrieval, open-source framework integration, and tailored development services to support a wide range of applications from NLP tasks to advanced automation."
"""
display(Markdown(out))


Aurelio AI is an AI company focused on language AI with several key areas of focus:

1. **Language Processing**: 
   - Develops tools and platforms for natural language processing (NLP) tasks such as translation, text generation, summarization, and more.

2. **AI Platform Development**:
   - Offers a centralized environment where engineers can develop, test, and deploy AI models efficiently, including features like real-time collaboration and automated testing.

3. **Information Retrieval**:
   - Manages data retrieval tasks effectively, optimizing search algorithms and indexing large datasets for quick information access.

4. **Open-Source Framework Integration**:
   - Utilizes frameworks like Semantic Router and Semantic Chunkers to enhance their AI Platform's capabilities in processing and understanding text data through components such as tokenization, parsing, and semantic analysis.

5. **Tailored Development Services**:
   - Provides organizations with customizations of their platforms and integration opportunities for new technologies, enabling them to bring advanced AI solutions to market effectively.

In summary, Aurelio AI focuses on language processing, AI platform development, information retrieval, open-source framework integration, and tailored development services to support a wide range of applications from NLP tasks to advanced automation."


## Chain-of-Thought Prompting

To implement CoT we don't need any specific LangChain objects, instead we are simply modifying how we instruct our LLM within the system prompt. We will ask the LLM to list the problems that need to be solved, to solve each problem individually, and then to arrive at the final answer.

In [20]:
cot_system_prompt = """
Be a helpful assistant and answer the user's question.

To answer the question, you must:

- List systematically and in precise detail all
  subproblems that need to be solved to answer the
  question.
- Solve each sub problem INDIVIDUALLY and in sequence.
- Finally, use everything you have worked through to
  provide the final answer.

Context: {context}
"""

cot_prompt_template = ChatPromptTemplate.from_messages([
    ("system", cot_system_prompt),
    ("user", "{query}"),
])

cot_pipeline = cot_prompt_template | llm

In [22]:
cot_result = cot_pipeline.invoke({"query": query, "context": context}).content
cot_result

"<think>\nOkay, so I need to figure out what Aurelio AI does based on the provided context. Let me start by reading through the information carefully.\n\nThe context mentions that Aurelio AI is an AI company focused on language AI. They have a strong expertise in building AI agents and have a solid background in information retrieval. They are behind several open-source frameworks like Semantic Router and Semantic Chunkers, and they also offer an AI Platform to engineers for tooling. Additionally, they provide development services to other organizations to help them bring their AI tech to market.\n\nNow, the user asked me what Aurelio AI does. I need to break this down into subproblems that would be necessary to answer. Let's think about the key areas of Aurelio AI based on the context provided.\n\nFirst, since they are focused on language AI and have expertise in building AI agents, it makes sense to mention their work with the LangChain framework as a significant part of what they do

In [23]:
output = """
Aurelio AI is an AI company focused on language AI, with expertise in building AI agents and a strong background in information retrieval. They are behind several open-source frameworks like Semantic Router and Semantic Chunkers and offer an AI Platform aimed at engineers. Here's a structured breakdown of what Aurelio AI does:\n\n1. **Language AI Development**: \n   - Focuses on natural language processing (NLP) tasks, including text and speech recognition.\n   - Utilizes open-source frameworks such as Semantic Router and Semantic Chunkers to contribute to the AI community.\n\n2. **LangChain Expertise**:\n   - Became LangChain Experts in September 2024 after a long track record, indicating their expertise in integrating LangChain into their work.\n\n3. **AI Platform for Engineers**:\n   - Provides tooling services tailored for engineers to help them build with AI technology effectively.\n\nThese subproblems highlight Aurelio AI's areas of focus and contributions within the field of language AI."
"""
display(Markdown(output))


Aurelio AI is an AI company focused on language AI, with expertise in building AI agents and a strong background in information retrieval. They are behind several open-source frameworks like Semantic Router and Semantic Chunkers and offer an AI Platform aimed at engineers. Here's a structured breakdown of what Aurelio AI does:

1. **Language AI Development**: 
   - Focuses on natural language processing (NLP) tasks, including text and speech recognition.
   - Utilizes open-source frameworks such as Semantic Router and Semantic Chunkers to contribute to the AI community.

2. **LangChain Expertise**:
   - Became LangChain Experts in September 2024 after a long track record, indicating their expertise in integrating LangChain into their work.

3. **AI Platform for Engineers**:
   - Provides tooling services tailored for engineers to help them build with AI technology effectively.

These subproblems highlight Aurelio AI's areas of focus and contributions within the field of language AI."
